In [193]:
import csv
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from IPython.display import Image
import seaborn as sns

In [2]:
# pandas display settings

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

In [304]:
# Scrape the job listings with Selenium
import time
from selenium import webdriver
 
driver = webdriver.Chrome('C:/Users/Arina/Chromedriver/chromedriver')  

In [285]:
filepath = '..\data_local\jobhunt_zapier_3.csv'
df = pd.read_csv(filepath)
df

,email_from,email_subject,email_content,date
0,We Work Remotely,"Listings for Sun, Feb 18",Your daily We Work Remotely job listings\r\n\r...,"Sun, 18 Feb 2018 04:14:19 +0000 (UTC)"
1,We Work Remotely,"Listings for Mon, Feb 19",Your daily We Work Remotely job listings\r\n\r...,"Mon, 19 Feb 2018 04:15:13 +0000 (UTC)"
2,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 23 o...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)"
3,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Mon, 19 Feb 2018 17:27:47 -0600 (CST)"
4,LinkedIn Job Alerts,Arina: 503 new jobs for 'data science',LinkedIn | 503 new jobs matching your search f...,"Tue, 20 Feb 2018 01:01:02 +0000 (UTC)"
5,We Work Remotely,"Listings for Tue, Feb 20",Your daily We Work Remotely job listings\r\n\r...,"Tue, 20 Feb 2018 04:21:59 +0000 (UTC)"
6,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 9 of...,"Tue, 20 Feb 2018 13:51:08 -0600 (CST)"
7,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Tue, 20 Feb 2018 19:23:41 -0600 (CST)"
8,LinkedIn Job Alerts,Arina: 776 new jobs for 'data science',LinkedIn | 776 new jobs matching your search f...,"Wed, 21 Feb 2018 02:12:09 +0000 (UTC)"
9,We Work Remotely,"Listings for Wed, Feb 21",Your daily We Work Remotely job listings\r\n\r...,"Wed, 21 Feb 2018 04:44:32 +0000 (UTC)"


In [286]:
df.describe()

,email_from,email_subject,email_content,date
count,14,14,14,14
unique,4,12,14,14
top,Indeed Job Alert,30+ new data science jobs,Recommended Jobs for You\r\n\r\nJobs 1 to 12 o...,"Wed, 21 Feb 2018 04:44:32 +0000 (UTC)"
freq,4,3,1,1


In [287]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
email_from       14 non-null object
email_subject    14 non-null object
email_content    14 non-null object
date             14 non-null object
dtypes: object(4)
memory usage: 528.0+ bytes


In [288]:
df.email_content

0     Your daily We Work Remotely job listings\r\n\r...
1     Your daily We Work Remotely job listings\r\n\r...
2     Recommended Jobs for You\r\n\r\nJobs 1 to 23 o...
3     Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...
4     LinkedIn | 503 new jobs matching your search f...
5     Your daily We Work Remotely job listings\r\n\r...
6     Recommended Jobs for You\r\n\r\nJobs 1 to 9 of...
7     Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...
8     LinkedIn | 776 new jobs matching your search f...
9     Your daily We Work Remotely job listings\r\n\r...
10    Recommended Jobs for You\r\n\r\nJobs 1 to 12 o...
11    Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...
12    LinkedIn | 951 new jobs matching your search f...
13    Recommended Jobs for You\r\n\r\nJobs 1 to 10 o...
Name: email_content, dtype: object

In [289]:
df.email_content[0]

"Your daily We Work Remotely job listings\r\n\r\nWe Work Remotely \r\n( https://u4898190.ct.sendgrid.net/wf/click?upn=YW1lRBveHigy9pfKCwINryPH9gXS1fDvrLx7LFjeI5kRW9biOb4VS8blYCFvZFezLJCeRUrwFMUmHXtbkjCLxg-3D-3D_f5qCWurvD1qQCAM8njv-2BcIZbstIiWZIpZOOjXCrIEXIJoYOgVu57VeQiqJS5T-2FmXFT9asgrRJwxK8pvVnsimT-2B3sVrNXnakqW2S-2B9Hi9q-2B4jvlnrVbuA5R7Ze2u1EN0-2BNlsqo9-2FTvMGAxDEapp43khJDYf1AsKI8cdT0GBebEsk-2F-2FGolqx16867NtPQPfzm1xvt8qwotcN2-2Bcv3KuCaQDQ-3D-3D )\r\n\r\n********************\r\nToday's job listings\r\n********************\r\n\r\nHere are your We Work Remotely job listings for Sunday, February\r\n18 2018.\r\n\r\n-----------\r\nProgramming\r\n-----------\r\n\r\nCloud and Rails Security Engineer at Retail Zipline\r\n\r\n( https://u4898190.ct.sendgrid.net/wf/click?upn=YW1lRBveHigy9pfKCwINryPH9gXS1fDvrLx7LFjeI5lll12hZfj-2B5jeSAhXevx0OKlrZdyS3nhmbSLQazzGBBDU4XuYL2dHviVQDSzt-2Fb4QlQRNK0mgZLH7XripqkQea_f5qCWurvD1qQCAM8njv-2BcIZbstIiWZIpZOOjXCrIEXIJoYOgVu57VeQiqJS5T-2FmXPtfJ7Z7Qo9aLC07ctpX1BF

In [290]:
df.email_content[0]

"Your daily We Work Remotely job listings\r\n\r\nWe Work Remotely \r\n( https://u4898190.ct.sendgrid.net/wf/click?upn=YW1lRBveHigy9pfKCwINryPH9gXS1fDvrLx7LFjeI5kRW9biOb4VS8blYCFvZFezLJCeRUrwFMUmHXtbkjCLxg-3D-3D_f5qCWurvD1qQCAM8njv-2BcIZbstIiWZIpZOOjXCrIEXIJoYOgVu57VeQiqJS5T-2FmXFT9asgrRJwxK8pvVnsimT-2B3sVrNXnakqW2S-2B9Hi9q-2B4jvlnrVbuA5R7Ze2u1EN0-2BNlsqo9-2FTvMGAxDEapp43khJDYf1AsKI8cdT0GBebEsk-2F-2FGolqx16867NtPQPfzm1xvt8qwotcN2-2Bcv3KuCaQDQ-3D-3D )\r\n\r\n********************\r\nToday's job listings\r\n********************\r\n\r\nHere are your We Work Remotely job listings for Sunday, February\r\n18 2018.\r\n\r\n-----------\r\nProgramming\r\n-----------\r\n\r\nCloud and Rails Security Engineer at Retail Zipline\r\n\r\n( https://u4898190.ct.sendgrid.net/wf/click?upn=YW1lRBveHigy9pfKCwINryPH9gXS1fDvrLx7LFjeI5lll12hZfj-2B5jeSAhXevx0OKlrZdyS3nhmbSLQazzGBBDU4XuYL2dHviVQDSzt-2Fb4QlQRNK0mgZLH7XripqkQea_f5qCWurvD1qQCAM8njv-2BcIZbstIiWZIpZOOjXCrIEXIJoYOgVu57VeQiqJS5T-2FmXPtfJ7Z7Qo9aLC07ctpX1BF

# Remotive emails

In [291]:
# \r\n\r\n-----------------------------------------------------------------\r\nTechnical Customer Support and Customer Success (\r\nhttps://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/zlvughn/aHR0cDovL3JlbW90aXZlLmlvL2pvYi90ZWNobmljYWwtY3VzdG9tZXItc3VwcG9ydC1jdXN0b21lci1zdWNjZXNzLw== )\r\n-----------------------------------------------------------------\r\n\r\n

#re_str = "(\r\n\r\n-*\r\n\w* \(\r\n)https://\w*(\)\r\n-*\r\n\r\n)"
#p = re.compile(re_str, re.IGNORECASE)

p =r"(?<=---).{1,100} \( ?https://\S* \)-*"


In [283]:
text = df.email_content[0]
text = re.sub(r'\\n', r'\r\n', text)
text = re.sub(r'\r\n', r'', text)
#text = re.sub(r'\n\n', r'', text)
text = re.sub(r'unsubscribe.*', r'', text, flags=re.IGNORECASE)
text = re.sub(r'.*-Rodolphe', r'', text, flags=re.IGNORECASE)
#print(text)
text

"Your daily We Work Remotely job listingsWe Work Remotely ( https://u4898190.ct.sendgrid.net/wf/click?upn=YW1lRBveHigy9pfKCwINryPH9gXS1fDvrLx7LFjeI5kRW9biOb4VS8blYCFvZFezLJCeRUrwFMUmHXtbkjCLxg-3D-3D_f5qCWurvD1qQCAM8njv-2BcIZbstIiWZIpZOOjXCrIEXIJoYOgVu57VeQiqJS5T-2FmXFT9asgrRJwxK8pvVnsimT-2B3sVrNXnakqW2S-2B9Hi9q-2B4jvlnrVbuA5R7Ze2u1EN0-2BNlsqo9-2FTvMGAxDEapp43khJDYf1AsKI8cdT0GBebEsk-2F-2FGolqx16867NtPQPfzm1xvt8qwotcN2-2Bcv3KuCaQDQ-3D-3D )********************Today's job listings********************Here are your We Work Remotely job listings for Sunday, February18 2018.-----------Programming-----------Cloud and Rails Security Engineer at Retail Zipline( https://u4898190.ct.sendgrid.net/wf/click?upn=YW1lRBveHigy9pfKCwINryPH9gXS1fDvrLx7LFjeI5lll12hZfj-2B5jeSAhXevx0OKlrZdyS3nhmbSLQazzGBBDU4XuYL2dHviVQDSzt-2Fb4QlQRNK0mgZLH7XripqkQea_f5qCWurvD1qQCAM8njv-2BcIZbstIiWZIpZOOjXCrIEXIJoYOgVu57VeQiqJS5T-2FmXPtfJ7Z7Qo9aLC07ctpX1BFdLkO0wsDmz7pVrr5A5yzlUXdyuswjLlW79dBHcScnJ2RiQRCcDXaHYh8m-2Fm6MCQRsB6J6N

In [268]:
links_list = re.findall(p, text)
links_list

[]

In [248]:
# should be 10

len(links_list)

0

In [239]:
# clean the links

links_list_clean = []
for linky in links_list:
    linky_clean = re.search(r'(?<=\() ?https?://.*(?=\))', linky, re.IGNORECASE).group().strip()
    if linky_clean not in set(links_list_clean):
        links_list_clean.append(linky_clean)
    
links_list_clean

['https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/e5uph7/aHR0cHM6Ly9yZW1vdGl2ZS5pby9qb2IvamF2YS1lbmdpbmVlci1zb2xyLw==',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/zvtghn/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9kZXZlbG9wZXItZXhwZXJpZW5jZS11c2luZy1zd2lmdC8=',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/60aeh8/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9iYWNrZW5kLWRldmVsb3Blci8=',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/2ou2ho/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9mcm9udGVuZC1kZXZlbG9wZXIv',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/93tzhn/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9sZWFkLXJlYWN0LWVuZ2luZWVyLw==',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/43avhe/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9tYXJrZXRpbmctZW5naW5lZXIv',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/0luwh9/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9zaXRlLXJlbGlhYmlsaXR5LWVuZ2luZWVyLTEv',
 'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/lotehm/aHR0cDovL3JlbW90aXZlLmlvL2pvYi9jb25zdWx0aW5nLWVuZ2l

In [109]:
len(links_list_clean)

10

In [132]:
# loop over job posts in email
linky_url = links_list_clean[0]
linky_url

'https://el2.convertkit-mail2.com/c/8kuo6xnw8foh3qe4p/e5uph7/aHR0cHM6Ly9yZW1vdGl2ZS5pby9qb2IvamF2YS1lbmdpbmVlci1zb2xyLw=='

In [133]:
driver.get(linky_url)

In [137]:
# job title
selector = '//h2[@class="page-title"]'
job_title = driver.find_elements_by_xpath(selector)[0].text
print(job_title)

Java Engineer (Solr)


In [117]:
# company name
selector = '//ul[@class="job-listing-meta meta"]/li[@class="job-company"]/a'
company_name = driver.find_element_by_xpath(selector).text
print(company_name)

Help Scout


In [119]:
# location
selector = '//ul[@class="job-listing-meta meta"]/li[@class="location"]'
location = driver.find_element_by_xpath(selector).text
print(location)

Remote


In [120]:
# date posted
selector = '//ul[@class="job-listing-meta meta"]/li[@class="date-posted"]'
date_posted = driver.find_element_by_xpath(selector).text
print(date_posted)

Posted 5 days ago


In [127]:
# job posting
selector = "//div[contains(concat(' ', @class, ' '), ' job-overview ')]"
job_posting = driver.find_element_by_xpath(selector).text
print(job_posting)

Overview
The ideal engineer for this position has experience designing and implementing search and large data solutions at scale. You love building RESTful APIs and other web services. You relish the opportunity to take on a challenging problem, choose the right tool for the job and build an elegant, scalable solution. Your work as an experienced Java engineer will empower over 8,000 businesses around the world to provide great customer service, and will support the other senior engineers on your team.
  We leverage Solr for several services at Help Scout and are looking for someone that can improve and maintain our implementation as we grow.
  Tools We Use
Apache Solr, Java, Spring, Play, MySQL, AWS, Hibernate
Slack, Trello, Github, Sentry, Postman Pro, Jenkins
About the role
You will join our two-person search team in owning the enhancement of the data quality and data processing on our large-scale search platform. On a daily basis, you’ll work on improving existing code, adding new 

In [128]:
# company description
selector = "//div[contains(concat(' ', @class, ' '), ' job-company-about ')]"
company_descr = driver.find_element_by_xpath(selector).text
print(company_descr)

About Help Scout
Help Scout is made by roughly 70 people in 40+ cities around the world, all with a passion for excellence in their craft.
We come from diverse backgrounds and are united by an enthusiasm for great products and delightful customer experiences.
Since launching in April of 2011, Help Scout now powers 8,000+ support teams in over 140 countries. Our software is used and trusted by businesses of every shape and size.
Designed to go the distance
Our mission is to foster the world’s customer-centric businesses by providing tools that serve customers in the most human, helpful way. While growth is important, team and values are paramount. We’re always striving to do impactful work we’re proud of in lieu of chasing overnight success.
We’re builders without borders
Our entire team is free to work remotely from wherever they choose. We overcome remote challenges through transparency, trust, and video conferencing. The reward is worth the effort—we’ll take talent, skill, and divers

In [129]:
# Get current link
driver.current_url

'https://remotive.io/job/java-engineer-solr/'

In [130]:
driver.close()

# Indeed emails

In [138]:
df

,email_from,email_subject,email_content,date
0,Rodolphe Dutel,Remotive Jobs Weekly,"Happy Friday,\r\n\r\nYou've asked us to send y...",2018-02-16 12:48:44
1,We Work Remotely,"Listings for Sun, Feb 18",Your daily We Work Remotely job listings\r\n\r...,"Sun, 18 Feb 2018 04:14:19 +0000 (UTC)"
2,We Work Remotely,"Listings for Mon, Feb 19",Your daily We Work Remotely job listings\r\n\r...,"Mon, 19 Feb 2018 04:15:13 +0000 (UTC)"
3,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 23 o...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)"


In [170]:
df.email_content[3]

"Recommended Jobs for You\r\n\r\nJobs 1 to 23 of 23 recommendations\r\nView all recommended jobs: https://www.indeed.com/recommendedjobs?from=rje&tk=1c6nnh4em1db5a8e&key=424055209985802272&rgtk=1c6nnnfhq39vpecv\r\n\r\n\r\nSenior Data Scientist\r\nBluecore - New York, NY\r\nWe are looking for Senior Data Scientists with strong mathematical backgrounds to work alongside our engineering teams to build the next generation of retail...\r\nBluecore - Feb 19\r\nhttps://www.indeed.com/rc/clk?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e\r\n\r\nData Scientist\r\nLoadsmart - New York, NY\r\nSoftware development skills in Python or R. At Loadsmart, we won't settle for the status quo in logistics....\r\nLoadSmart - Feb 19\r\nhttps://www.indeed.com/rc/clk?jk=59a797491eb26a8c&from=rje&rgtk=1c6nnh4em1db5a8e\r\n\r\nData Scientist\r\nFoursquare - New York, NY\r\n3+ years work experience as a data scientist or in analytics, working with big datasets. Familiarity with Foursquare or location data and

In [171]:
# https://www.indeed.com/viewjob?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e
df.email_from[3].strip().lower()

'indeed job alert'

In [172]:
text = df.email_content[3]
text = re.sub(r'\r\n', r' ', text)
text = re.sub(r'Cancel this Recommended Job Alert:.*', r'', text, flags=re.IGNORECASE)
text

"Recommended Jobs for You  Jobs 1 to 23 of 23 recommendations View all recommended jobs: https://www.indeed.com/recommendedjobs?from=rje&tk=1c6nnh4em1db5a8e&key=424055209985802272&rgtk=1c6nnnfhq39vpecv   Senior Data Scientist Bluecore - New York, NY We are looking for Senior Data Scientists with strong mathematical backgrounds to work alongside our engineering teams to build the next generation of retail... Bluecore - Feb 19 https://www.indeed.com/rc/clk?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e  Data Scientist Loadsmart - New York, NY Software development skills in Python or R. At Loadsmart, we won't settle for the status quo in logistics.... LoadSmart - Feb 19 https://www.indeed.com/rc/clk?jk=59a797491eb26a8c&from=rje&rgtk=1c6nnh4em1db5a8e  Data Scientist Foursquare - New York, NY 3+ years work experience as a data scientist or in analytics, working with big datasets. Familiarity with Foursquare or location data and an ability to clearly... foursquare - Feb 18 https://www.in

In [173]:
# parse out the raw link parts
p =r"https://www.indeed.com/rc/\S* "

links_list = re.findall(p, text)
links_list

['https://www.indeed.com/rc/clk?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=59a797491eb26a8c&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=7490c513ac10a12c&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=4eceac18fce041d1&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=6b82f29c6741d6e5&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=89a671ea11d57c12&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=1c9df0f237366ecc&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=21cec82830aaeac2&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=0c3d5016781c6555&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=4a3134853e77264f&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=e9af3391c7e1069f&from=rje&rgtk=1c6nnh4em1db5a8e ',
 'https://www.indeed.com/rc/clk?jk=44971a3c0fc6d1db&fr

In [174]:
len(links_list)

23

In [175]:
# loop over job posts in email
linky_url = links_list[0]
linky_url

'https://www.indeed.com/rc/clk?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e '

In [176]:
driver.get(linky_url)

In [149]:
# job title
selector = '//table[@id="job-content"]//b[@class="jobtitle"]'
job_title = driver.find_elements_by_xpath(selector)[0].text
print(job_title)

Senior Data Scientist


In [150]:
# company name
selector = '//table[@id="job-content"]//span[@class="company"]'
company_name = driver.find_elements_by_xpath(selector)[0].text
print(company_name)

Bluecore


In [151]:
# location
selector = '//table[@id="job-content"]//span[@class="location"]'
location = driver.find_elements_by_xpath(selector)[0].text
print(location)

New York, NY 10002


In [157]:
# date posted
#selector = '//div[@id="info-link-row"]//span[@class="date"]'
selector = '//span[@class="date"]'
date_posted = driver.find_elements_by_xpath(selector)[0].text
print(date_posted)

2 days ago


In [158]:
# job posting
selector = '//span[@id="job_summary"]'
job_posting = driver.find_elements_by_xpath(selector)[0].text
print(job_posting)

We are looking for Senior Data Scientists with strong mathematical backgrounds to work alongside our engineering teams to build the next generation of retail and commerce models that delight and empower marketers. The ideal candidate is one that has several years of experience researching, building, serving, and maintaining data science models at scale. They have first-hand experience with what works and what doesn’t, and are eager to share this experience with more junior members and guide them through that process. They are also able to and excited to help architect and build out the data science architecture needed to accelerate innovation on models and facilitate serving and maintaining them. Finally, they should be curious and eager to identify and explore the myriad of other products that can be built on our data asset. Our culture emphasizes making good tradeoffs, working as a team, and leaving your ego at the door.
First-party data is at the core of everything we build and the 

In [177]:
driver.current_url

'https://www.indeed.com/viewjob?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e'

In [178]:
# Get company description
name_selector = '//div[@class= "cmp_info"]//div[@class= "cmp_title"]/a'
driver.find_element_by_xpath(name_selector).click()
current_url = driver.current_url

In [179]:
current_url

'https://www.indeed.com/cmp/Bluecore?from=vjcmpinfo&fromjk=6c7ebde0dbd89142&attributionid=vjcmpinfo'

In [180]:
# short company descr
selector = '//span[@id="cmp-short-description"]'
company_descr1 = driver.find_elements_by_xpath(selector)[0].text
print(company_descr1)

At Bluecore we are transforming the way eCommerce marketers use data and automation to communicate with customers. Bluecore’s customer experience platform is designed to simplify the process of ingesting terabytes of behavioral data and automatically taking action on precise insights, driving engagement and conversion rates that defy industry standards.


In [167]:
from bs4 import BeautifulSoup

In [181]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <title>\n   Bluecore Careers and Employment | Indeed.com\n  </title>\n  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>\n  <meta content="115882278440564" property="fb:app_id"/>\n  <meta content="website" property="og:type"/>\n  <meta content="Indeed.com" property="og:site_name"/>\n  <meta content="http://labkit.indeed.com/static/indeed_fb.png" property="og:image"/>\n  <meta content="What it is like to work at Bluecore" property="og:title"/>\n  <meta content="Take an inside look at working for Bluecore as told by real employees.  Find reviews, frequently asked questions and open jobs on Indeed.com." property="og:description"/>\n  <meta content="https://www.indeed.com/cmp/Bluecore" property="og:url"/>\n  <meta content="Research and review Bluecore jobs. Learn more about a career with Bluecore including all recent jobs, hiring trends, salaries, work environment and more." name="description"/>\n 

In [226]:
selector = '//span[@id="cmp-more-description"]'

company_descr2 = str(soup.findAll('span',attrs={'id':'cmp-more-description'})[0])
company_descr2

'<span class="hidden" id="cmp-more-description">We\'re one of New York City’s fastest growing SaaS start-ups, working with more than 190 customers representing more than 250 high-end apparel, electronics, automotive and other consumer brands.<br/><br/>We have only one rule: use good judgment. We hire incredible people so we don’t have to institute cumbersome rules or processes. We want you to be free to think, create and get things done on your own terms. We love positive energy in the office. In fact, we need it—happy people are more efficient and creative. So if 6 PM improv classes or midday gym sessions are what makes you happy, we will support you. When you join us, you’re stuck with us. We hired you for you, not just for this job. We know the best people move up and often go on to start their own companies. We want to help you get where you want to go.<br/><br/>Perks &amp; Benefits<br/>- Highly competitive compensation package and opportunity to work for one of the fastest growing

In [230]:
if soup.findAll('span',attrs={'id':'cmp-more-description'}):
    print('test')

test


In [224]:
company_descr3 = re.sub(r'<a ?.* id="cmp-less-link">less</a>', r'', company_descr2, flags=re.IGNORECASE)
company_descr3 = re.sub(r'<span ?.* id="cmp-more-description">', r'', company_descr3, flags=re.IGNORECASE)
company_descr3 = re.sub(r'</span>', r'', company_descr3, flags=re.IGNORECASE)
company_descr3 = re.sub(r'(<br>)|(<br/>)', r'\r\n', company_descr3, flags=re.IGNORECASE)
company_descr3 = re.sub(r'&amp;', r'and', company_descr3, flags=re.IGNORECASE)

print(len(company_descr3))
print(company_descr3)

1479
We're one of New York City’s fastest growing SaaS start-ups, working with more than 190 customers representing more than 250 high-end apparel, electronics, automotive and other consumer brands.

We have only one rule: use good judgment. We hire incredible people so we don’t have to institute cumbersome rules or processes. We want you to be free to think, create and get things done on your own terms. We love positive energy in the office. In fact, we need it—happy people are more efficient and creative. So if 6 PM improv classes or midday gym sessions are what makes you happy, we will support you. When you join us, you’re stuck with us. We hired you for you, not just for this job. We know the best people move up and often go on to start their own companies. We want to help you get where you want to go.

Perks and Benefits
- Highly competitive compensation package and opportunity to work for one of the fastest growing start-ups in New York City.
- Laid-back office, casual attire. Je

In [164]:
# more company descr
name_selector = '//a[@id= "cmp-more-link"]'
driver.find_element_by_xpath(name_selector).click()

selector = '//span[@id="cmp-more-description"]'
company_descr2 = driver.find_elements_by_xpath(selector)[0].text
print(company_descr2)

We're one of New York City’s fastest growing SaaS start-ups, working with more than 190 customers representing more than 250 high-end apparel, electronics, automotive and other consumer brands.

We have only one rule: use good judgment. We hire incredible people so we don’t have to institute cumbersome rules or processes. We want you to be free to think, create and get things done on your own terms. We love positive energy in the office. In fact, we need it—happy people are more efficient and creative. So if 6 PM improv classes or midday gym sessions are what makes you happy, we will support you. When you join us, you’re stuck with us. We hired you for you, not just for this job. We know the best people move up and often go on to start their own companies. We want to help you get where you want to go.

Perks & Benefits
- Highly competitive compensation package and opportunity to work for one of the fastest growing start-ups in New York City.
- Laid-back office, casual attire. Jeans and

In [166]:
company_descr = company_descr1 + ' ' + company_descr2
company_descr

"At Bluecore we are transforming the way eCommerce marketers use data and automation to communicate with customers. Bluecore’s customer experience platform is designed to simplify the process of ingesting terabytes of behavioral data and automatically taking action on precise insights, driving engagement and conversion rates that defy industry standards. We're one of New York City’s fastest growing SaaS start-ups, working with more than 190 customers representing more than 250 high-end apparel, electronics, automotive and other consumer brands.\n\nWe have only one rule: use good judgment. We hire incredible people so we don’t have to institute cumbersome rules or processes. We want you to be free to think, create and get things done on your own terms. We love positive energy in the office. In fact, we need it—happy people are more efficient and creative. So if 6 PM improv classes or midday gym sessions are what makes you happy, we will support you. When you join us, you’re stuck with u

## Dates cleaning 

In [233]:
from datetime import datetime

datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
datetime_object

datetime.datetime(2005, 6, 1, 13, 33)

In [234]:
datetime_object = datetime.strptime('Sun, 18 Feb 2018 04:14:19 +0000 (UTC)', '%a, %d %b %Y %H:%M:%S %z (%Z)')
datetime_object

datetime.datetime(2018, 2, 18, 4, 14, 19, tzinfo=datetime.timezone(datetime.timedelta(0), 'UTC'))

In [272]:
print(datetime_object)

2018-02-18 04:14:19+00:00


## LinkedIn

In [319]:
driver = webdriver.Chrome('C:/Users/Arina/Chromedriver/chromedriver')  

In [292]:
df

,email_from,email_subject,email_content,date
0,We Work Remotely,"Listings for Sun, Feb 18",Your daily We Work Remotely job listings\r\n\r...,"Sun, 18 Feb 2018 04:14:19 +0000 (UTC)"
1,We Work Remotely,"Listings for Mon, Feb 19",Your daily We Work Remotely job listings\r\n\r...,"Mon, 19 Feb 2018 04:15:13 +0000 (UTC)"
2,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 23 o...,"Mon, 19 Feb 2018 13:12:58 -0600 (CST)"
3,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Mon, 19 Feb 2018 17:27:47 -0600 (CST)"
4,LinkedIn Job Alerts,Arina: 503 new jobs for 'data science',LinkedIn | 503 new jobs matching your search f...,"Tue, 20 Feb 2018 01:01:02 +0000 (UTC)"
5,We Work Remotely,"Listings for Tue, Feb 20",Your daily We Work Remotely job listings\r\n\r...,"Tue, 20 Feb 2018 04:21:59 +0000 (UTC)"
6,Indeed Job Alert,You have new recommended jobs: Senior Data Sci...,Recommended Jobs for You\r\n\r\nJobs 1 to 9 of...,"Tue, 20 Feb 2018 13:51:08 -0600 (CST)"
7,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Tue, 20 Feb 2018 19:23:41 -0600 (CST)"
8,LinkedIn Job Alerts,Arina: 776 new jobs for 'data science',LinkedIn | 776 new jobs matching your search f...,"Wed, 21 Feb 2018 02:12:09 +0000 (UTC)"
9,We Work Remotely,"Listings for Wed, Feb 21",Your daily We Work Remotely job listings\r\n\r...,"Wed, 21 Feb 2018 04:44:32 +0000 (UTC)"


In [293]:
df.email_content[4]

"LinkedIn | 503 new jobs matching your search for data science\r\n----------------------------------------------------------------\r\n\r\nData Science Analyst\r\nC2R Ventures\r\nGreater New York City Area\r\nInApply\r\nView job: https://www.linkedin.com/comm/jobs/view/563721832?alertAction=markasviewed&savedSearchAuthToken=1%26AQGD7Cf3Tt6dAwAAAWGwujktWL8C_I4kHFD7zV1BRz9ufqZvsBSHLA0y4K5f5vR9KgmKyCtbJXPjYGLrXS_DiNjc77kP8W6K7KMpu2Uw9Rh3EErZFaOx_NzvkRsNB-_MWJC5SUZFfjKN-4LQp08-_MFPyIrRvq56aswAyMhBwkMAaOhI6NDB8-WAD62uh4RdHI6QOPQuc62QMgybsb6JYIpbS27yQTtrMLHnBhiJrGHuMT8zFG_Zx2V2n2JTwTI986VccYLMBWhMGJHRwqEA21nSiviBU6ddyuGrtS9CqOk7cRJvWTDcJg%26AeWO7EqIvT-dT7KVC4fnKBSUV6Zt&savedSearchId=280713974&refId=64f7ef24-e228-4d80-a08f-63558a66e077&trk=eml-job-alert-member-details&midToken=AQHbEzDWyKpnOA&trkEmail=eml-email_job_alert_single_02-null-4-null-null-ct6d3%7Ejduy1mzd%7Euk-null-jobs%7Eview&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_single_02%3BYBms%2FIJUT9aLyjWi4kVMuQ%3D%3D\r\n\r\n---------------

In [320]:
# https://www.indeed.com/viewjob?jk=6c7ebde0dbd89142&from=rje&rgtk=1c6nnh4em1db5a8e
df.email_from[4].strip().lower()

'linkedin job alerts'

In [321]:
text = df.email_content[4]
#text = re.sub(r'\r\n', r' ', text)
#text = re.sub(r'Cancel this Recommended Job Alert:.*', r'', text, flags=re.IGNORECASE)
text

"LinkedIn | 503 new jobs matching your search for data science\r\n----------------------------------------------------------------\r\n\r\nData Science Analyst\r\nC2R Ventures\r\nGreater New York City Area\r\nInApply\r\nView job: https://www.linkedin.com/comm/jobs/view/563721832?alertAction=markasviewed&savedSearchAuthToken=1%26AQGD7Cf3Tt6dAwAAAWGwujktWL8C_I4kHFD7zV1BRz9ufqZvsBSHLA0y4K5f5vR9KgmKyCtbJXPjYGLrXS_DiNjc77kP8W6K7KMpu2Uw9Rh3EErZFaOx_NzvkRsNB-_MWJC5SUZFfjKN-4LQp08-_MFPyIrRvq56aswAyMhBwkMAaOhI6NDB8-WAD62uh4RdHI6QOPQuc62QMgybsb6JYIpbS27yQTtrMLHnBhiJrGHuMT8zFG_Zx2V2n2JTwTI986VccYLMBWhMGJHRwqEA21nSiviBU6ddyuGrtS9CqOk7cRJvWTDcJg%26AeWO7EqIvT-dT7KVC4fnKBSUV6Zt&savedSearchId=280713974&refId=64f7ef24-e228-4d80-a08f-63558a66e077&trk=eml-job-alert-member-details&midToken=AQHbEzDWyKpnOA&trkEmail=eml-email_job_alert_single_02-null-4-null-null-ct6d3%7Ejduy1mzd%7Euk-null-jobs%7Eview&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_single_02%3BYBms%2FIJUT9aLyjWi4kVMuQ%3D%3D\r\n\r\n---------------

In [322]:
# parse out the raw link parts
p =r"https://www.linkedin.com/comm/jobs/view/\S*"

links_list = re.findall(p, text)
links_list

['https://www.linkedin.com/comm/jobs/view/563721832?alertAction=markasviewed&savedSearchAuthToken=1%26AQGD7Cf3Tt6dAwAAAWGwujktWL8C_I4kHFD7zV1BRz9ufqZvsBSHLA0y4K5f5vR9KgmKyCtbJXPjYGLrXS_DiNjc77kP8W6K7KMpu2Uw9Rh3EErZFaOx_NzvkRsNB-_MWJC5SUZFfjKN-4LQp08-_MFPyIrRvq56aswAyMhBwkMAaOhI6NDB8-WAD62uh4RdHI6QOPQuc62QMgybsb6JYIpbS27yQTtrMLHnBhiJrGHuMT8zFG_Zx2V2n2JTwTI986VccYLMBWhMGJHRwqEA21nSiviBU6ddyuGrtS9CqOk7cRJvWTDcJg%26AeWO7EqIvT-dT7KVC4fnKBSUV6Zt&savedSearchId=280713974&refId=64f7ef24-e228-4d80-a08f-63558a66e077&trk=eml-job-alert-member-details&midToken=AQHbEzDWyKpnOA&trkEmail=eml-email_job_alert_single_02-null-4-null-null-ct6d3%7Ejduy1mzd%7Euk-null-jobs%7Eview&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_single_02%3BYBms%2FIJUT9aLyjWi4kVMuQ%3D%3D',
 'https://www.linkedin.com/comm/jobs/view/603794044?alertAction=markasviewed&savedSearchAuthToken=1%26AQGD7Cf3Tt6dAwAAAWGwujktWL8C_I4kHFD7zV1BRz9ufqZvsBSHLA0y4K5f5vR9KgmKyCtbJXPjYGLrXS_DiNjc77kP8W6K7KMpu2Uw9Rh3EErZFaOx_NzvkRsNB-_MWJC5SUZFfjKN-4LQp

In [323]:
len(links_list)

11

In [324]:
df.date[4]

'Tue, 20 Feb 2018 01:01:02 +0000 (UTC)'

In [325]:
df.email_from[4]

'LinkedIn Job Alerts'

In [326]:
# loop over job posts in email
linky_url = links_list[0]
linky_url

'https://www.linkedin.com/comm/jobs/view/563721832?alertAction=markasviewed&savedSearchAuthToken=1%26AQGD7Cf3Tt6dAwAAAWGwujktWL8C_I4kHFD7zV1BRz9ufqZvsBSHLA0y4K5f5vR9KgmKyCtbJXPjYGLrXS_DiNjc77kP8W6K7KMpu2Uw9Rh3EErZFaOx_NzvkRsNB-_MWJC5SUZFfjKN-4LQp08-_MFPyIrRvq56aswAyMhBwkMAaOhI6NDB8-WAD62uh4RdHI6QOPQuc62QMgybsb6JYIpbS27yQTtrMLHnBhiJrGHuMT8zFG_Zx2V2n2JTwTI986VccYLMBWhMGJHRwqEA21nSiviBU6ddyuGrtS9CqOk7cRJvWTDcJg%26AeWO7EqIvT-dT7KVC4fnKBSUV6Zt&savedSearchId=280713974&refId=64f7ef24-e228-4d80-a08f-63558a66e077&trk=eml-job-alert-member-details&midToken=AQHbEzDWyKpnOA&trkEmail=eml-email_job_alert_single_02-null-4-null-null-ct6d3%7Ejduy1mzd%7Euk-null-jobs%7Eview&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_single_02%3BYBms%2FIJUT9aLyjWi4kVMuQ%3D%3D'

In [327]:
driver.get(linky_url)

In [328]:
# job title
selector = '//section[@id="top-card"]//h1[@class="title"]'
job_title = driver.find_elements_by_xpath(selector)[0].text
print(job_title)

Data Science Analyst


In [329]:
# company name
selector = '//section[@id="top-card"]//span[@class="company"]'
company_name = driver.find_elements_by_xpath(selector)[0].text
print(company_name)

C2R Ventures


In [330]:
# location
selector = '//section[@id="top-card"]//h3[@class="location"]/span/span'
location = driver.find_elements_by_xpath(selector)[0].text
print(location)

Greater New York City Area


In [331]:
# date posted
#selector = '//div[@id="info-link-row"]//span[@class="date"]'
selector = '//section[@id="top-card"]//ul[@class="posting-info"]/li[@class="posted"]/span'
date_posted = driver.find_elements_by_xpath(selector)[0].text
print(date_posted)

Posted 3 days ago


In [332]:
# job posting
selector = '//section[@id="summary-detail"]//div[@class="description-section"]'
job_posting = driver.find_elements_by_xpath(selector)[0].text
print(job_posting)

Innovative Health Services organization is seeking a Data Science Analyst for key role within their Population Health Analytics team.
 
In this role you’ll work collaboratively with stakeholders from across the organization to develop sophisticated analytics to provide information, insights and BI (Business Intelligence) solutions that contribute to sound strategic planning, decision-making, goal setting, and effective performance measurement.  

This is a Fulltime role located in New York City.
    Required Domain Skills:
Minimum 2 years of Data Science/Analytics Development experience for a Health Provider, Health Plan or Accountable Care Organization including either:
Working knowledge of a health care EMR such as Epic/Clarity, eCW, etc.; a payer claims system such as Facets, Amisys, etc.; or a hospital/provider billing system such as Idx, Soarian, etc.
    Required Technical Skills:
Strong data mining skills using tools like SQL, R, Python, SAS, Java/MapReduce, Aster, etc.
Strong a

In [337]:
driver.current_url

'https://www.linkedin.com/comm/jobs/view/563721832?alertAction=markasviewed&savedSearchAuthToken=1%26AQGD7Cf3Tt6dAwAAAWGwujktWL8C_I4kHFD7zV1BRz9ufqZvsBSHLA0y4K5f5vR9KgmKyCtbJXPjYGLrXS_DiNjc77kP8W6K7KMpu2Uw9Rh3EErZFaOx_NzvkRsNB-_MWJC5SUZFfjKN-4LQp08-_MFPyIrRvq56aswAyMhBwkMAaOhI6NDB8-WAD62uh4RdHI6QOPQuc62QMgybsb6JYIpbS27yQTtrMLHnBhiJrGHuMT8zFG_Zx2V2n2JTwTI986VccYLMBWhMGJHRwqEA21nSiviBU6ddyuGrtS9CqOk7cRJvWTDcJg%26AeWO7EqIvT-dT7KVC4fnKBSUV6Zt&savedSearchId=280713974&refId=64f7ef24-e228-4d80-a08f-63558a66e077&trk=eml-job-alert-member-details&midToken=AQHbEzDWyKpnOA&trkEmail=eml-email_job_alert_single_02-null-4-null-null-ct6d3%7Ejduy1mzd%7Euk-null-jobs%7Eview&lipi=urn%3Ali%3Apage%3Aemail_email_job_alert_single_02%3BYBms%2FIJUT9aLyjWi4kVMuQ%3D%3D'

In [338]:
# company descr
selector = '//section[@id="company-lcp"]//div[@class="description-container"]'
company_descr = driver.find_elements_by_xpath(selector)[0].text
print(company_descr)

C2R Ventures is a Talent Advisory Collective dedicated to disrupting the Technology, Financial, FinTech and Management Consulting recruitment space.

Our values and our approach are simple: relationships (both client and candidate) matter above all else.

Our core mission is to represent elite candidates and connect them with businesses who are using Mathematics and Technology as their driving force and competitive advantage.
